In [5]:
!module list


Currently Loaded Modules:
  1) gcc/7.4.0   2) tensorflow/1.14.0

 

>

In [15]:
!cat /users/htoivone/.local/share/jupyter/kernels/venv/kernel.json

{
 "argv": [
  "/users/htoivone/keras-bert-ner/venv/bin/python",
  "-m",
  "ipykernel_launcher",
  "-f",
  "{connection_file}"
 ],
 "display_name": "venv",
 "language": "python"
}

In [17]:
!pip list | grep keras-bert

keras-bert                       0.86.0   


In [2]:
!pip install keras-bert | head -3

ERROR: Pipe to stdout was broken
Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='UTF-8'>
BrokenPipeError: [Errno 32] Broken pipe


In [18]:
!source venv/bin/activate

In [19]:
import keras_bert

ModuleNotFoundError: No module named 'keras_bert'

In [18]:
from bert import tokenization
from keras_bert import load_trained_model_from_checkpoint, AdamWarmup

ModuleNotFoundError: No module named 'keras_bert'

!module list

In [7]:
!pip install -r requirements.txt


  Using cached https://files.pythonhosted.org/packages/44/e1/dc0757b20b56c980b5553c1b5c4c32d378c7055ab7bfa92006801ad359ab/Keras-2.4.3-py2.py3-none-any.whl
  Found existing installation: Keras 2.2.4
    Not uninstalling keras at /appl/soft/ai/miniconda3/envs/tensorflow-1.14.0/lib/python3.7/site-packages, outside environment /users/htoivone/keras-bert-ner/venv
    Can't uninstall 'Keras'. No files were found to uninstall.


In [6]:
!pwd

/users/htoivone/keras-bert-ner


In [ ]:
def load_pretrained(options):
    model = load_trained_model_from_checkpoint(
        options.bert_config_file,
        options.init_checkpoint,
        training=False,
        trainable=True,
        seq_len=options.max_seq_length,
    )
    tokenizer = tokenization.FullTokenizer(
        vocab_file=options.vocab_file,
        do_lower_case=options.do_lower_case
    )
    return model, tokenizer

In [ ]:
from argparse import Namespace

args = Namespace(
    batch_size=4, 
    bert_config_file='/users/htoivone/links/august/scripts/../models/bert-base-finnish-cased-v1/bert_config.json',
    dev_data=None, 
    do_lower_case=False, 
    early_stopping=False, 
    init_checkpoint='/users/htoivone/links/august/scripts/../models/bert-base-finnish-cased-v1/bert_model.ckpt', 
    learning_rate=5e-05, 
    max_seq_length=128, 
    ner_model_dir='/users/htoivone/links/august/scripts/../ner-models/turku-ner2-model', 
    num_train_epochs=4, 
    output_file='output.tsv', 
    test_data='/users/htoivone/links/august/scripts/../data/AnatEM-1.0.2/conll_single_class//test.tsv', 
    train_data='/users/htoivone/links/august/scripts/../data/AnatEM-1.0.2/conll_single_class//train.tsv', 
    viterbi=False, 
    vocab_file='/users/htoivone/links/august/scripts/../models/bert-base-finnish-cased-v1/vocab.txt', 
    warmup_proportion=0.1)

In [ ]:
def main(argv):
    argparser = argument_parser()
    args = argparser.parse_args(argv[1:])
    seq_len = args.max_seq_length    # abbreviation

    #print(args)
    #exit(1)
    pretrained_model, tokenizer = load_pretrained(args)

    train_words, train_tags = read_conll(args.train_data)
    test_words, test_tags = read_conll(args.test_data)
    train_data = process_sentences(train_words, train_tags, tokenizer, seq_len)
    test_data = process_sentences(test_words, test_tags, tokenizer, seq_len)

    label_list = get_labels(train_data.labels)
    tag_map = { l: i for i, l in enumerate(label_list) }
    inv_tag_map = { v: k for k, v in tag_map.items() }

    init_prob, trans_prob = viterbi_probabilities(train_data.labels, tag_map)

    train_x = encode(train_data.combined_tokens, tokenizer, seq_len)
    test_x = encode(test_data.combined_tokens, tokenizer, seq_len)

    train_y, train_weights = label_encode(
        train_data.combined_labels, tag_map, seq_len)
    test_y, test_weights = label_encode(
        test_data.combined_labels, tag_map, seq_len)

    ner_model = create_ner_model(pretrained_model, len(tag_map))
    optimizer = create_optimizer(len(train_x[0]), args)

    ner_model.compile(
        optimizer,
        loss='sparse_categorical_crossentropy',
        sample_weight_mode='temporal',
        metrics=['sparse_categorical_accuracy']
    )

    callbacks_list = [
    keras.callbacks.EarlyStopping(monitor='val_loss',patience=1,),
    keras.callbacks.ModelCheckpoint(filepath=args.ner_model_dir+'/model.hdf5',
    monitor='val_loss',save_best_only=True,)]

    if args.early_stopping:
        print("NER:Using early stopping.")
        ner_model.fit(
        train_x,train_y,sample_weight=train_weights,epochs=args.num_train_epochs,
        batch_size=args.batch_size,callbacks=callbacks_list,validation_data=(test_x,test_y))
    else:
        ner_model.fit(train_x,train_y,sample_weight=train_weights,
        epochs=args.num_train_epochs,batch_size=args.batch_size,)

    if args.ner_model_dir is not None:
        label_list = [v for k, v in sorted(list(inv_tag_map.items()))]
        save_ner_model(ner_model, tokenizer, label_list, args)
        save_viterbi_probabilities(init_prob, trans_prob, inv_tag_map, args)

    probs = ner_model.predict(test_x, batch_size=args.batch_size)
    preds = np.argmax(probs, axis=-1)

    pred_tags = []
    for i, pred in enumerate(preds):
        pred_tags.append([inv_tag_map[t]
                          for t in pred[1:len(test_data.tokens[i])+1]])

    lines = write_result(
        args.output_file, test_data.words, test_data.lengths,
        test_data.tokens, test_data.labels, pred_tags
    )

    c = conlleval.evaluate(lines)
    conlleval.report(c)
    return 0


if __name__ == '__main__':
    sys.exit(main(sys.argv))
